## Before training

This program saves the last 3 generations of models to Google Drive. Since 1 generation of models is >1GB, you should have at least 3GB of free space in Google Drive. If you do not have such free space, it is recommended to create another Google Account.

Training requires >10GB VRAM. (T4 should be enough) Inference does not require such a lot of VRAM.

## Installation

In [ ]:
#@title Check GPU
!nvidia-smi



Tue May 23 11:50:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


In [ ]:
#@title Install dependencies
#@markdown pip may fail to resolve dependencies and raise ERROR, but it can be ignored.
!python -m pip install -U pip wheel
%pip install -U ipython 

#@markdown Branch (for development)
BRANCH = "none" #@param {"type": "string"}
if BRANCH == "none":
    %pip install -U so-vits-svc-fork
else:
    %pip install -U git+https://github.com/34j/so-vits-svc-fork.git@{BRANCH}

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.7/797.7 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.0 MB/s eta 0:00:00
  Attempting uninstall: ipython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipython-7.34.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 8.13.2 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.3/91.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 91.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 106.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 678.1/678.1 kB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 103.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 29.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Prepari

In [ ]:
from types import NotImplementedType

#@title Creating a set
source_dir = "/content/gdrive/MyDrive/SPEECH_COMBINED" #@param {type:"string"}
intermediate_dir = f'/content/gdrive/MyDrive/Gothic Multispeaker addon'
target_dir = "dataset_raw" #@param {type:"string"}
character_file = "/content/gdrive/MyDrive/Other/polish/adam bauman/train_filelist.txt"
directory = "Other/polish" #@param {type:"string"}
save_directory = "trening" #@param {type:"string"}
model_directory = "logs" #@param {type:"string"}
parent_dir = "/content"
variable_name = ""
import os
import shutil

def create_intermediate_directory(source_dir, character_filename_dict,character_list):
  character_key = None

  for character in character_list:
    for key,value_list in character_filename_dict.items():
      if character in value_list:
        character_key = key

        # print(character_key, character)
        character_dir = os.path.join(intermediate_dir,f'{character}')
        if not os.path.exists(character_dir):
          search_string = f'{character}'  
          with open(character_key, 'r') as f:
            lines = f.readlines()
          matching_lines = []
          for line in lines:
            first_slash_index = line.find('/')
            first_pipe_index = line.find('|')
            stripped_line = line[first_slash_index+1:first_pipe_index]
            if search_string in stripped_line:
              matching_lines.append(stripped_line)

          for filename in os.listdir(source_dir):
            print(filename)
            if filename in matching_lines:
              filepath = os.path.join(source_dir, filename)
              if not os.path.exists(character_dir):
                !mkdir -p "{character_dir}"
              shutil.copy(filepath, character_dir)


def create_collab_directory_and_preprocess(source_dir,target_dir):
  if not os.path.exists(target_dir):
    !mkdir -p "{target_dir}"
    !cp -R "{source_dir}"/* "{target_dir}"/
    !svc pre-resample
    !svc pre-config 
    !svc pre-hubert -fm crepe

def save_on_gdrive(save_directory,parent_dir,all_in_directory):
  if all_in_directory == False:
    if not os.path.exists(save_directory):
      !mkdir -p gdrive/MyDrive/so-vits-svc-fork/"{save_directory}"
    !cp -r configs gdrive/MyDrive/so-vits-svc-fork/"{save_directory}"
    !cp -r dataset gdrive/MyDrive/so-vits-svc-fork/"{save_directory}"
    !cp -r dataset_raw gdrive/MyDrive/so-vits-svc-fork/"{save_directory}"
    !cp -r filelists gdrive/MyDrive/so-vits-svc-fork/"{save_directory}"
  else:
    if os.path.commonprefix([parent_dir, 'dataset']) == parent_dir:
      !cp -r gdrive/MyDrive/so-vits-svc-fork/"{save_directory}"/dataset /content
    if os.path.commonprefix([parent_dir, 'dataset_raw']) == parent_dir:
      !cp -r gdrive/MyDrive/so-vits-svc-fork/"{save_directory}"/dataset_raw /content
    if os.path.commonprefix([parent_dir, 'filelists']) == parent_dir:
      !cp -r gdrive/MyDrive/so-vits-svc-fork/"{save_directory}"/filelists /content
    if os.path.commonprefix([parent_dir, 'configs']) == parent_dir:
      !cp -r gdrive/MyDrive/so-vits-svc-fork/"{save_directory}"/configs /content

if directory != "":
  directory = directory + '/'

training_files = {'1':'dataset',
             '2':'dataset_raw',
             '3':'filelists',
             '4':'configs'}

character_filename_dict = {
    f'/content/gdrive/MyDrive/{directory}andrzej tomecki/train_filelist.txt' : ["GRIMES", "GRAVO", "TORREZ", "NEFARIUS", "GILBERT", "TORLOF", "POCK", "GREG","LOTHAR"],
    f'/content/gdrive/MyDrive/{directory}adam bauman/train_filelist.txt' : ['DIEGO',
'LARES',
'MORDRAG',
'GOMEZ',
'ŚWISTAK',
'GORNATOTH',
'BAALTYON',
'DARRION',
'DRAKE',
'OKYL'
],
    f'/content/gdrive/MyDrive/{directory}jacek kopczynski/train_filelist.txt' : ['LEE',
'COR ANGAR',
'CRONOS',
'BLOODWYN',
'BALORO',
'GORHANIS',
'GORBOBA',
'JARVIS',
'SCAR',
'CUTTER'
],
    f'/content/gdrive/MyDrive/{directory}jaroslaw domin/train_filelist.txt' : ["YBERION",
"CAVALORN",
"RICELORD",
"BAALORUN",
"FISK",
"SKIP",
"BARTHOLO",
"ALBERTO",
"CIPHER",
"RODRIGUEZ"
],
    f'/content/gdrive/MyDrive/{directory}piotr bak/train_filelist.txt' : ["IAN",
"SKORPION",
"BAALLUKOR",
"BAALKAGAN",
"BAALTONDRAL",
"GORNARAN",
"GORNAKOSH",
"GORNAVID",
"AIDAN",
"ARTO",
"CAINE",
"DRAGO",
"PACHO",
"BUTCH"
],

    f'/content/gdrive/MyDrive/{directory}jacek soltysiak/train_filelist.txt' : ["NYRAS",
"BAALISIDRO",
"BAALNETBEK",
"GUY",
"JESSE",
"DUSTY",
"MELVIN"
],

f'/content/gdrive/MyDrive/{directory}lukasz nowicki/train_filelist.txt' : ["SNAF",
"SCATTY",
"HEREK",
"HARLOK",
"BALOR",
"GARP",
"SENYAN"],
f'/content/gdrive/MyDrive/{directory}tomasz marzecki/train_filelist.txt' : ["XARDAS",
"SATURAS",
"CORRISTO",
"RIORDIAN",
"DAMAROK",
"CORD"
],

f'/content/gdrive/MyDrive/{directory}przemyslaw nikiel/train_filelist.txt' : ["GORN",
"THORUS",
"WILK",
"GORNADRAK",
"GORNABAR",
"HORACY",
"AARON",
"HUNO",
"SWINEY",
"KHARIM"
],

f'/content/gdrive/MyDrive/{directory}pawel szczesny/train_filelist.txt' : ["LESTER",
"FINGERS",
"KLIN",
"BAALTARAN",
"FORTUNO",
"ALEPH",
"KIRGO"
],
f'/content/gdrive/MyDrive/{directory}robert tondera/train_filelist.txt' : ["MUD",
"LEFTY",
"SHRIKE",
"JACKAL",
"SNIPES",
"ULBERT",
"VIRAN",
"JORU",
"RATFORD",
"SANTINO"
],
f'/content/gdrive/MyDrive/{directory}janusz wituch/train_filelist.txt' : ["KALOM",
"RAVEN",
"STT",
"DEXTER",
"BAALPARVEZ",
"KYLE",
"ROSCOE",
"SHARKY"
],
f'/content/gdrive/MyDrive/{directory}kinga ilgner/train_filelist.txt' : [
"CASSIA",
"GRITTA",
"ELENA",
"SARA",
"KATI",
"BARBERA",
"NADJA",
"SONJA"
]

}
# character_list = ["CASSIA",
# "GRITTA",
# "ELENA",
# "SARA",
# "KATI",
# "BARBERA",
# "NADJA",
# "RAVEN",
# "STT",
# "DEXTER",
# "BAALPARVEZ",
# "KYLE",
# "ROSCOE",
# "LEFTY",
# "JACKAL",
# "SNIPES",
# "ULBERT",
# "VIRAN",
# "JORU",
# "RATFORD",
# "FINGERS",
# "KLIN",
# "BAALTARAN",
# "FORTUNO",
# "ALEPH",
# "KIRGO","THORUS",
# "WILK",
# "GORNADRAK",
# "HORACY",
# "AARON",
# "HUNO",
# "SWINEY",
# "KHARIM","SATURAS",
# "CORRISTO",
# "RIORDIAN",
# "CORD","SCATTY",
# "HEREK",
# "HARLOK",
# "BALOR",
# "BAALISIDRO",
# "GUY",
# "JESSE",
# "DUSTY",
# "MELVIN","SKORPION",
# "BAALLUKOR",
# "BAALKAGAN",
# "BAALTONDRAL",
# "GORNARAN",
# "AIDAN",
# "CAINE",
# "DRAGO",
# "PACHO",
# "CAVALORN",
# "RICELORD",

# "FISK",
# "SKIP",
# "BARTHOLO",
# "ALBERTO",
# "CIPHER",
# 'COR ANGAR',
# 'CRONOS',
# 'BLOODWYN',
# 'BALORO',
# 'JARVIS',
# 'SCAR',
# 'CUTTER','LARES',
# 'MORDRAG',
# 'GOMEZ',
# 'ŚWISTAK',
# 'DRAKE',
# 'OKYL']

character_list = ['XARDAS','MUD',"GRIMES", "GRAVO", "TORREZ", "NEFARIUS", "GILBERT", "TORLOF", "POCK", "GREG","LOTHAR","YBERION","IAN"]

check_dir = os.path.join("/content/gdrive/MyDrive/so-vits-svc-fork/",save_directory)
all_in_check_dir = True

for key in training_files:
  if not os.path.exists(check_dir):
    all_in_check_dir = False
    break
  else:
    value = training_files[key]
    if value not in os.listdir(check_dir):
      all_in_check_dir = False
      break

if all_in_check_dir == True:
  save_on_gdrive(save_directory,parent_dir,all_in_check_dir)
  %load_ext tensorboard
  %tensorboard --logdir gdrive/MyDrive/so-vits-svc-fork/"{save_directory}"/"{model_directory}"
  !svc train --model-path gdrive/MyDrive/so-vits-svc-fork/"{save_directory}"/"{model_directory}"
else:                                         #brak czegos z training_files w check_dir
  create_intermediate_directory(source_dir,character_filename_dict,character_list)
  create_collab_directory_and_preprocess(intermediate_dir, target_dir)
  save_on_gdrive(save_directory,parent_dir,all_in_check_dir)
  %load_ext tensorboard
  %tensorboard --logdir gdrive/MyDrive/so-vits-svc-fork/"{save_directory}"/"{model_directory}"
  !svc train --model-path gdrive/MyDrive/so-vits-svc-fork/"{save_directory}"/"{model_directory}"

Streaming output truncated to the last 5000 lines.
INFO_SATURAS_BOOK_14_03.WAV
DIA_SENYAN_JUSTTALK_15_00.WAV
INFO_GORN_RUINWAIT_09_02.WAV
SVM_1_HEKILLEDHIM.WAV
GRD_277_GARDIST_SITTINGORK_INFO_06_03.WAV
DIA_NYRAS_ORT_03_01.WAV
ORG_862_JACKO_GUARD_INFOWERT_INFO_15_00.WAV
PC_PSIONIC_SOON_INFO_15_01.WAV
KDW_600_SATURAS_KDWAUFNAHME_INFO_15_07.WAV
DIA_HEREK_BULLY_01_00.WAV
INFO_GRD_7_DIELAGE_07_01.WAV
DIA_NYRAS_ORT_03_02.WAV
INFO_MINE_GRD_13_DIELAGE_15_00.WAV
DIA_CAVALORN_LEHRER_SCHLEICHEN_12_02.WAV
INFO_ORCSHAMAN_IDOL_17_05.WAV
SVM_11_YOUDEFEATEDNOV_GUARD.WAV
DIA_WEDGE_LEHRER_15_00.WAV
DIA_MILTEN_WOTORREZ_02_01.WAV
INFO_SFB_1_DASLAGER_15_00.WAV
INFO_CRONOS_KALOM_08_03.WAV
INFO_MINE_VLK_3_DIELAGE_15_00.WAV
INFO_XARDAS_DISTURB_14_03.WAV
INFO_DIEGO_BRIEF_15_00.WAV
DIA_FLETCHER_WONEK_06_02.WAV
SVM_12_YOUKILLEDNCFOLK.WAV
INFO_THORUS_PERM2_09_01.WAV
INFO_DIEGO_TEACH_BACK_11_02.WAV
DIA_HUNO_HOWSYOURBUSINESS_15_00.WAV
INFO_THORUS_READYFORGOMEZ_15_02.WAV
SVM_4_YOUKILLEDONEOFUS.WAV
SIT_2_PSI_YBERION_

<IPython.core.display.Javascript object>

/bin/bash: svc: command not found


In [ ]:
import shutil

shutil.rmtree('/content/dataset_raw')

In [ ]:
# import os
# somedir = '/content/gdrive/MyDrive/Gothic Multispeaker'
# for directory in os.listdir(somedir):
#   file_count = os.path.join(somedir,directory)
#   counter = 0
#   for files in os.listdir(file_count):
#     counter+=1
#   print(counter,file_count)
#   3 /content/gdrive/MyDrive/Gothic Multispeaker/SHARKY
# 1 /content/gdrive/MyDrive/Gothic Multispeaker/SHRIKE
# 3 /content/gdrive/MyDrive/Gothic Multispeaker/DAMAROK
# 3 /content/gdrive/MyDrive/Gothic Multispeaker/SONJA
# 1 /content/gdrive/MyDrive/Gothic Multispeaker/SANTINO
# 2 /content/gdrive/MyDrive/Gothic Multispeaker/SENYAN
# 3 /content/gdrive/MyDrive/Gothic Multispeaker/ARTO
# 1 /content/gdrive/MyDrive/Gothic Multispeaker/BUTCH
# 1 /content/gdrive/MyDrive/Gothic Multispeaker/RODRIGUEZ
# 2 /content/gdrive/MyDrive/Gothic Multispeaker/DARRION
import os
import shutil
miko = f'/content/gdrive/MyDrive/Other/polish/janusz wituch/train_filelist.txt'
character_key = None

for line in miko:
        # print(character_key, character)
    with open(miko, 'r') as f:
        lines = f.readlines()
    search_string = "KALOM"
    matching_lines = []
    for line in lines:
        first_slash_index = line.find('/')
        first_pipe_index = line.find('|')
        stripped_line = line[first_slash_index+1:first_pipe_index]
        if search_string in stripped_line:
            matching_lines.append(stripped_line)
    for filename in os.listdir('/content/gdrive/MyDrive/SPEECH_COMBINED'):
        print(filename)
        if filename in matching_lines:
            filepath = os.path.join('/content/gdrive/MyDrive/SPEECH_COMBINED', filename)
            shutil.copy(filepath, '/content/gdrive/MyDrive/politycy/bezi')


Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
INFO_FINDNPC_ST_CORANGAR_08_01.WAV
SVM_13_YOUSTOLEFROMUS.WAV
INFO_CORKALOM_BRINGBOOK_SUCCESS_10_03.WAV
DIA_SLD_753_BALORO_WORUMGEHTS_INFO_08_02.WAV
INFO_XARDASDEMON_INTRO_15_05.WAV
GUR_1201_CORKALOM_WEG_INFO_10_03.WAV
INFO_SATURAS_BOOK_14_03.WAV
DIA_SENYAN_JUSTTALK_15_00.WAV
INFO_GORN_RUINWAIT_09_02.WAV
SVM_1_HEKILLEDHIM.WAV
GRD_277_GARDIST_SITTINGORK_INFO_06_03.WAV
DIA_NYRAS_ORT_03_01.WAV
ORG_862_JACKO_GUARD_INFOWERT_INFO_15_00.WAV
PC_PSIONIC_SOON_INFO_15_01.WAV
KDW_600_SATURAS_KDWAUFNAHME_INFO_15_07.WAV
DIA_HEREK_BULLY_01_00.WAV
INFO_GRD_7_DIELAGE_07_01.WAV
DIA_NYRAS_ORT_03_02.WAV
INFO_MINE_GRD_13_DIELAGE_15_00.WAV
DIA_CAVALORN_LEHRER_SCHLEICHEN_12_02.WAV
INFO_ORCSHAMAN_IDOL_17_05.WAV
SVM_11_YOUDEFEATEDNOV_GUARD.WAV
DIA_WEDGE_LEHRER_15_00.WAV
DIA_MILTEN_WOTORREZ_02_01.WAV
INFO_SFB_1_DASLAGER_15_00.WAV
INFO_CRONOS_KALOM_08_03.WAV
INFO_MINE_VLK_3_DIELAGE_15_00.WAV
INFO_XARDAS_DISTURB_14_03.WAV
INFO_DIEGO_BRIEF_15_00.WAV
D

In [ ]:
import os
import shutil
for file in os.listdir('/content/gdrive/MyDrive/SPEECH_COMBINED'):
    if "KALOM" in file:
        print(file)
        path = os.path.join('/content/gdrive/MyDrive/SPEECH_COMBINED',file)
        shutil.copy(path,'/content/gdrive/MyDrive/politycy/KALOM')

GUR_1201_CORKALOM_SACHE_INFO_15_01.WAV
GUR_1201_CORKALOM_JOINPSI_10_02.WAV
GUR_1201_CORKALOM_JOINPSI_NOT_10_02.WAV
GUR_1201_CORKALOM_WEG_INFO_15_01.WAV
MIS_1_PSI_KALOM_KRAUTBOTEBACK_10_02.WAV
INFO_FINDNPC_ST_CORKALOM_13_01.WAV
MIS_2_PSI_KALOM_BRINGMCQEGGS_SUCCESS_RUNE_15_01.WAV
INFO_CORKALOM_BELOHNUNG_SCROLL_10_01.WAV
GUR_1201_CORKALOM_RUN_INFO_15_01.WAV
ORG_826_MORDRAG_GOTOKALOMNOW_15_02.WAV
INFO_CORKALOM_BRINGBOOK_INFO_10_10.WAV
GUR_1201_CORKALOM_JOINPSI_15_03.WAV
GUR_1212_MADCORKALOM_TALK2SC_10_07.WAV
GUR_1201_CORKALOM_JOINPSI_10_11.WAV
GUR_1201_CORKALOM_VISION_INFO_10_04.WAV
DIA_GOMEZ_HELLO_KONTAKTE_KALOM_11_01.WAV
GUR_1201_CORKALOM_FIND_INFO_15_03.WAV
GUR_1212_MADCORKALOM_TALK2SC_10_08.WAV
MIS_1_PSI_KALOM_KRAUTBOTEBACK_15_00.WAV
INFO_CORKALOM_BRINGBOOK_INFO_10_02.WAV
GUR_1201_CORKALOM_JOINPSI_15_04.WAV
GUR_1201_CORKALOM_RAT_INFO_10_03.WAV
GUR_1201_CORKALOM_SACHE_NEIN_10_04.WAV
GUR_1201_CORKALOM_VISION_INFO_15_05.WAV
MIS_2_PSI_KALOM_BRINGMCQEGGS_SUCCESS_15_01.WAV
INFO_FINDNPC_ST_CO

In [ ]:
!zip -r "Gothic Multispeaker.zip" '/content/gdrive/MyDrive/Gothic Multispeaker'

Scanning files ...
  adding: content/gdrive/MyDrive/Gothic Multispeaker/ (stored 0%)
  adding: content/gdrive/MyDrive/Gothic Multispeaker/DIEGO/ (stored 0%)
  adding: content/gdrive/MyDrive/Gothic Multispeaker/DIEGO/DIA_ADDON_DIEGONW_WILLHELPYOU_11_02.WAV (deflated 13%)
  adding: content/gdrive/MyDrive/Gothic Multispeaker/DIEGO/DIA_ADDON_DIEGO_TOGETHER_11_01.WAV (deflated 27%)
  adding: content/gdrive/MyDrive/Gothic Multispeaker/DIEGO/DIA_ADDON_DIEGO_GOHOME_11_10.WAV (deflated 18%)
  adding: content/gdrive/MyDrive/Gothic Multispeaker/DIEGO/DIA_ADDON_DIEGO_TOGETHER_11_06.WAV (deflated 14%)
  adding: content/gdrive/MyDrive/Gothic Multispeaker/DIEGO/DIA_ADDON_DIEGO_TOOFAR_11_23.WAV (deflated 13%)
  adding: content/gdrive/MyDrive/Gothic Multispeaker/DIEGO/DIA_ADDON_DIEGO_TOOFAR_11_21.WAV (deflated 14%)
  adding: content/gdrive/MyDrive/Gothic Multispeaker/DIEGO/DIA_ADDON_DIEGO_TOOFAR_11_20.WAV (deflated 13%)
  adding: content/gdrive/MyDrive/Gothic Multispeaker/DIEGO/DIA_ADDON_DIEGO_TOGETHER

## Training

In [ ]:
b#@title Get the author's voice as a source
import random
NAME = str(random.randint(1, 49))
TYPE = "fsd50k" #@param ["", "digit", "dog", "fsd50k"]
CUSTOM_FILEPATH = "" #@param {type: "string"}
if CUSTOM_FILEPATH != "":
    NAME = CUSTOM_FILEPATH
else:
    # it is extremely difficult to find a voice that can download from the internet directly
    if TYPE == "dog":
        !wget -N f"https://huggingface.co/datasets/437aewuh/dog-dataset/resolve/main/dogs/dogs_{NAME:.0000}.wav" -O {NAME}.wav
    elif TYPE == "digit":
        # george, jackson, lucas, nicolas, ...
        !wget -N f"https://github.com/Jakobovski/free-spoken-digit-dataset/raw/master/recordings/0_george_{NAME}.wav" -O {NAME}.wav
    elif TYPE == "fsd50k":
        !wget -N f"https://huggingface.co/datasets/Fhrozen/FSD50k/blob/main/clips/dev/{10000+int(NAME)}.wav" -O {NAME}.wav
    else:
        !wget -N f"https://zunko.jp/sozai/utau/voice_{"kiritan" if NAME < 25 else "itako"}{NAME % 5 + 1}.wav" -O {NAME}.wav
from IPython.display import Audio, display
display(Audio(f"{NAME}.wav"))

In [ ]:
#@title Use trained model
#@markdown **Put your .wav file in `so-vits-svz
from IPython.display import Audio, display
# NAME = '/content/gdrive/MyDrive/so-vits-svc-fork/X2Download.app - No jak tam jebane psy polskie kurwa TFU_! (128 kbps).mp3'
# display(Audio(f"{NAME}",rate=22050))
!svc infer '/content/gdrive/MyDrive/so-vits-svc-fork/psy.mp3' -m /content/gdrive/MyDrive/G_4.pth -c /content/gdrive/config.json -s 0 -o /content/gdrive/MyDrive/test2.mp3
# display(Audio(f"drive/MyDrive/so-vits-svc-fork/audio/{NAME}.out.wav", autoplay=True,rate=22050))

[20:49:48] INFO     [20:49:48] Version: 3.14.1                    ]8;id=553930;file:///usr/local/lib/python3.10/dist-packages/so_vits_svc_fork/__main__.py\__main__.py]8;;\:]8;id=825375;file:///usr/local/lib/python3.10/dist-packages/so_vits_svc_fork/__main__.py#31\31]8;;\
Usage: svc infer [OPTIONS] INPUT_PATH
Try 'svc infer -h' for help.

Error: Invalid value for '-c' / '--config-path': Path '/content/gdrive/config.json' does not exist.


In [ ]:
!ffmpeg -i /content/gdrive/MyDrive/politycy/xxx -ss 00:26:00 -to 00:30:30 -c:v copy -c:a copy /content/gdrive/MyDrive/politycy/xxx.out.mp3

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [ ]:
!ffmpeg -i  /content/gdrive/MyDrive/politycy/xxx.out.mp3 -af silencedetect=n=-50dB:d=0.5 -f null - 2> /content/gdrive/MyDrive/politycy/output.txt
!grep -E "silence_start|silence_end"  /content/gdrive/MyDrive/politycy/output.txt | awk '{print $NF}' >  /content/gdrive/MyDrive/politycy/timestamps.txt
!ffmpeg -i /content/gdrive/MyDrive/politycy/kaczynski_out.mp3 -af "volume=enable='between(t,$(sed -n '1p' /content/gdrive/MyDrive/politycy/timestamps.txt) + 0.001, $(sed -n '2p' /content/gdrive/MyDrive/politycy/timestamps.txt) + 0.001) + between(t,$(sed -n '3p' /content/gdrive/MyDrive/politycy/timestamps.txt) + 0.001, $(sed -n '4p' /content/gdrive/MyDrive/politycy/timestamps.txt) + 0.001)':volume=0, volume=enable='between(t,$(sed -n '2p' /content/gdrive/MyDrive/politycy/timestamps.txt) + 0.001, $(sed -n '3p' /content/gdrive/MyDrive/politycy/timestamps.txt))':volume=1" /content/gdrive/MyDrive/politycy/kaczynski_output.mp3

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [ ]:
!ffmpeg -i /content/gdrive/MyDrive/politycy/kaczor.mp3 -f segment -segment_time 6 -c copy /content/gdrive/MyDrive/politycy/output_%03d.wav


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [ ]:
!unzip gdrive/MyDrive/morawlogs/morawlogs.zip -d  ~/XD

Archive:  gdrive/MyDrive/morawlogs/morawlogs.zip
   creating: /root/XD/home/ubuntu/logs/
   creating: /root/XD/home/ubuntu/logs/lightning_logs/
   creating: /root/XD/home/ubuntu/logs/lightning_logs/version_0/
  inflating: /root/XD/home/ubuntu/logs/lightning_logs/version_0/hparams.yaml  
  inflating: /root/XD/home/ubuntu/logs/lightning_logs/version_0/events.out.tfevents.1684619721.192-18-139-234.1663.0  
  inflating: /root/XD/home/ubuntu/logs/D_0.pth  
   creating: /root/XD/home/ubuntu/logs/44k/
  inflating: /root/XD/home/ubuntu/logs/44k/diego2.mp3  
  inflating: /root/XD/home/ubuntu/logs/44k/x.mp3  
  inflating: /root/XD/home/ubuntu/logs/44k/Recording2.mp3  
  inflating: /root/XD/home/ubuntu/logs/44k/pom.mp3  
  inflating: /root/XD/home/ubuntu/logs/44k/diego4.mp3  
  inflating: /root/XD/home/ubuntu/logs/44k/diego.mp3  
  inflating: /root/XD/home/ubuntu/logs/44k/mud.wav  
  inflating: /root/XD/home/ubuntu/logs/44k/diego1.mp3  
   creating: /root/XD/home/ubuntu/logs/44k/lightning_logs/
 